In [9]:
using DataFrames
using CSV
using DataFrames
using StatsBase

# Load the CSV files into data frames
train = CSV.read("../../data/train.csv", DataFrame)
test = CSV.read("../../data/test.csv", DataFrame)
data_dict = CSV.read("../../data/data_dictionary.csv", DataFrame)

# Display the data
first(train, 5)
first(test, 5)
first(data_dict, 5)

Row,Instrument,Field,Description,Type,Values,Value Labels
,String,String,String,String15,String31?,String?
1,Identifier,id,Participant's ID,str,missing,missing
2,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",missing
3,Demographics,Basic_Demos-Age,Age of participant,float,missing,missing
4,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
5,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",missing


In [7]:
# Specify the categorical columns
cat_c = ["Basic_Demos-Enroll_Season", "CGAS-Season", "Physical-Season", "Fitness_Endurance-Season", 
           "FGC-Season", "BIA-Season", "PAQ_A-Season", "PAQ_C-Season", "SDS-Season", "PreInt_EduHx-Season",
            "Basic_Demos-Sex", "FGC-FGC_CU_Zone", "FGC-FGC_GSND_Zone", "FGC-FGC_GSD_Zone",
           "FGC-FGC_PU_Zone", "FGC-FGC_SRL_Zone", "FGC-FGC_SRR_Zone", "FGC-FGC_TL_Zone", "BIA-BIA_Activity_Level_num",
           "BIA-BIA_Frame_num"]

id_col = ["id"]

pciat = filter(name -> startswith(name, "PCIAT-PCIAT"), names(train))
append!(pciat, ["sii", "PCIAT-Season"])

columns_float = setdiff(names(train), cat_c ∪ pciat ∪ id_col)

# Filter the numeric columns and convert them to numeric types
train_filtered_float = train[:, columns_float]
train_filtered_cat = train[:, cat_c]

for col in names(train_filtered_float)
    train_filtered_float[!, col] = map(x -> 
        try 
            parse(Float64, string(x)) 
        catch 
            missing 
        end, train_filtered_float[!, col])
end

In [10]:
function calculate_mode(column)
    nonmissing_values = skipmissing(column)
    return mode(nonmissing_values)
end

for col in names(train_filtered_cat)
    column = train_filtered_cat[!, col]
    mode_val = calculate_mode(column)
    train_filtered_cat[!, col] = map(x -> ismissing(x) ? mode_val : x, column)
end

In [11]:
lnr = IAI.OptKNNImputationLearner(method = "opt_knn", random_seed=12)

Unfitted OptKNNImputationLearner:
  random_seed: 12

In [12]:
IAI.fit!(lnr, train_filtered_float)

[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
┌ Warning: Interpretable AI license expires soon: 2024-12-31T00:00:00. If you need to renew, please send us the following machine ID:
└ bed1c049b33cf139e0b18189fbbed293e73573685741fed0de8b4df77326d308
[ Warning: Imputed value for X[35, 38] less than machine epsilon, has been rounded to zero.


Fitted OptKNNImputationLearner

In [13]:
completed_data = IAI.transform(lnr, train_filtered_float)

Row,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_GSND,FGC-FGC_GSD,FGC-FGC_PU,FGC-FGC_SRL,FGC-FGC_SRR,FGC-FGC_TL,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,5.0,51.0,16.8773,46.0,50.8,25.1759,65.7422,84.7461,111.762,4.80544,6.98867,29.8993,0.0,16.9722,17.6707,0.0,7.0,6.0,6.0,2.66855,16.8792,932.498,1492.0,8.25598,41.5862,13.8177,3.06143,9.21377,24.4349,8.89536,38.9177,19.5413,32.6909,2.0824,2.51022,38.7298,54.9038,3.0
2,9.0,69.6367,14.0356,48.0,46.0,22.0,75.0,70.0,122.0,5.4307,8.40915,19.9539,3.0,13.9922,14.6729,5.0,11.0,11.0,3.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,21.0352,14.974,39.4497,15.4107,27.0552,2.10036,2.34,46.0,64.0,0.0
3,10.0,71.0,16.6487,56.5,75.6,25.7451,65.0,94.0,117.0,5.0,7.0,33.0,20.0,10.2,14.7,7.0,10.0,10.0,5.0,3.88273,16.6285,1110.26,1754.93,16.0348,60.5203,13.9868,2.64169,11.6179,28.5858,15.8997,56.6375,28.9606,44.6206,2.10983,2.17,38.0,54.0,2.0
4,9.0,71.0,18.2923,56.0,81.6,27.2762,60.0,97.0,117.0,6.0,9.0,37.0,18.0,13.3037,14.8864,5.0,7.0,7.0,7.0,3.84191,18.2943,1131.43,1923.44,15.5925,62.7757,14.074,4.22033,18.8243,30.4041,16.779,58.9338,26.4798,45.9966,2.10493,2.451,31.0,45.0,0.0
5,18.0,59.6789,20.7594,64.3865,121.531,28.2087,66.7627,70.223,116.142,4.7217,6.91961,21.6813,10.6898,25.2675,27.3333,4.44271,10.2109,10.8322,11.6687,5.13932,21.7131,1399.49,2194.79,29.2667,91.3267,15.8664,5.84669,33.2904,38.0969,23.9631,86.1873,43.4592,67.3635,1.04,2.54648,45.0772,63.0468,1.99281
6,13.0,50.0,22.28,59.5,112.2,28.0867,60.0,73.0,102.0,5.13475,7.94598,15.0221,12.0,16.5,17.9,6.0,10.0,11.0,8.0,4.33036,30.1865,1330.97,1996.45,30.2124,84.0285,16.6877,13.4988,67.9715,32.9141,20.902,79.6982,35.3804,63.1265,2.02108,4.11,40.0,56.0,0.0
7,10.0,69.9724,19.6608,55.0,84.6,26.3617,123.0,83.0,163.0,4.61677,6.71143,29.7459,9.0,17.1003,18.8065,2.0,11.0,11.0,11.0,3.78271,19.6629,1135.86,1817.38,16.3275,63.247,14.7,4.96291,21.353,30.8936,16.0259,59.4643,26.1957,47.2211,2.10729,3.67,27.0,40.0,3.0
8,10.0,61.2465,16.8613,59.25,84.2,27.0,71.0,90.0,116.0,4.10185,5.69859,34.6195,0.0,12.6,11.1,0.0,0.0,0.0,4.0,4.05726,16.8631,1180.04,1888.06,21.94,67.9527,13.6092,3.25395,16.2474,28.5367,17.476,63.8954,28.768,50.4767,2.14851,1.27,38.5729,54.5234,2.0
9,15.0,66.5426,20.6969,65.4208,126.306,29.1629,66.5897,67.7032,117.417,4.84765,7.09307,21.5363,23.6469,30.5967,31.9185,8.40764,9.90091,10.1505,10.4988,5.12391,20.8834,1404.76,2552.08,27.7686,91.8881,15.0202,5.8632,34.977,38.6079,25.5118,86.7643,43.7555,66.3764,2.0332,3.05772,34.9918,50.1466,2.0


In [14]:
append!(cat_c, pciat, id_col)

44-element Vector{String}:
 "Basic_Demos-Enroll_Season"
 "CGAS-Season"
 "Physical-Season"
 "Fitness_Endurance-Season"
 "FGC-Season"
 "BIA-Season"
 "PAQ_A-Season"
 "PAQ_C-Season"
 "SDS-Season"
 "PreInt_EduHx-Season"
 "Basic_Demos-Sex"
 "FGC-FGC_CU_Zone"
 "FGC-FGC_GSND_Zone"
 ⋮
 "PCIAT-PCIAT_13"
 "PCIAT-PCIAT_14"
 "PCIAT-PCIAT_15"
 "PCIAT-PCIAT_16"
 "PCIAT-PCIAT_17"
 "PCIAT-PCIAT_18"
 "PCIAT-PCIAT_19"
 "PCIAT-PCIAT_20"
 "PCIAT-PCIAT_Total"
 "sii"
 "PCIAT-Season"
 "id"

In [15]:
# Combine the imputed data with the removed columns
train_final = hcat(completed_data, train[:, cat_c])
# Reorder columns to match the original dataset order
train_final = train_final[:, names(train)]

Row,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
,String15,String7,Float64?,Int64,String7?,Float64?,String7?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String7?,Float64?,Float64?,Float64?,String7?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,String7?,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,String7?,Float64?,String7?,Float64?,String7?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,String7?,Float64?,Float64?,String7?,Float64?,Int64?
1,00008ff9,Fall,5.0,0,Winter,51.0,Fall,16.8773,46.0,50.8,25.1759,65.7422,84.7461,111.762,missing,4.80544,6.98867,29.8993,Fall,0.0,0,16.9722,missing,17.6707,missing,0.0,0,7.0,0,6.0,0,6.0,1,Fall,2,2.66855,16.8792,932.498,1492.0,8.25598,41.5862,13.8177,3.06143,9.21377,1,24.4349,8.89536,38.9177,19.5413,32.6909,missing,2.0824,missing,2.51022,Fall,5,4,4,0,4,0,0,4,0,0,4,0,4,4,4,4,4,4,2,4,55,missing,38.7298,54.9038,Fall,3.0,2
2,000fd460,Summer,9.0,0,missing,69.6367,Fall,14.0356,48.0,46.0,22.0,75.0,70.0,122.0,missing,5.4307,8.40915,19.9539,Fall,3.0,0,13.9922,missing,14.6729,missing,5.0,0,11.0,1,11.0,1,3.0,0,Winter,2,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1,21.0352,14.974,39.4497,15.4107,27.0552,missing,2.10036,Fall,2.34,Fall,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fall,46.0,64.0,Summer,0.0,0
3,00105258,Summer,10.0,1,Fall,71.0,Fall,16.6487,56.5,75.6,25.7451,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1,10.2,1,14.7,2,7.0,1,10.0,1,10.0,1,5.0,0,missing,missing,3.88273,16.6285,1110.26,1754.93,16.0348,60.5203,13.9868,2.64169,11.6179,missing,28.5858,15.8997,56.6375,28.9606,44.6206,missing,2.10983,Summer,2.17,Fall,5,2,2,1,2,1,1,2,1,1,1,0,1,1,1,0,2,2,1,1,28,Fall,38.0,54.0,Summer,2.0,0
4,00115b9f,Winter,9.0,0,Fall,71.0,Summer,18.2923,56.0,81.6,27.2762,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1,13.3037,missing,14.8864,missing,5.0,0,7.0,0,7.0,0,7.0,1,Summer,3,3.84191,18.2943,1131.43,1923.44,15.5925,62.7757,14.074,4.22033,18.8243,2,30.4041,16.779,58.9338,26.4798,45.9966,missing,2.10493,Winter,2.451,Summer,4,2,4,0,5,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,Summer,31.0,45.0,Winter,0.0,1
5,0016bb22,Spring,18.0,1,Summer,59.6789,missing,20.7594,64.3865,121.531,28.2087,66.7627,70.223,116.142,missing,4.7217,6.91961,21.6813,missing,10.6898,missing,25.2675,missing,27.3333,missing,4.44271,missing,10.2109,missing,10.8322,missing,11.6687,missing,missing,missing,5.13932,21.7131,1399.49,2194.79,29.2667,91.3267,15.8664,5.84669,33.2904,missing,38.0969,23.9631,86.1873,43.4592,67.3635,Summer,1.04,missing,2.54

In [16]:

# Save the final DataFrame to a CSV file
CSV.write("../training_sets/imputed_train_optimal_knn.csv", train_final)

"../imputed_train_optimal_knn.csv"